# Because nobody wants to wait for pandas.

In [ ]:
from fair21 import FAIR
from fair21.interface import initialise, fill
from fair21.io import read_properties

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
species, properties = read_properties()

In [ ]:
f = FAIR(ch4_method='thornhill2021')
f.define_time(1750, 2500, 1)
f.define_configs(['unspecified'])
f.define_scenarios(scenarios)
f.define_species(species, properties)

In [ ]:
f.allocate()
f.fill_from_rcmip()

In [ ]:
f.emissions

In [ ]:
f.emissions.to_netcdf('../data/rcmip/ssp_emissions_fair2.1.nc')